In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from matplotlib import font_manager, rc
%matplotlib inline
import folium
from folium.plugins import MarkerCluster
from folium import FeatureGroup, LayerControl
from folium.features import GeoJson
from shapely.geometry import shape, Point
import json

# 글꼴 경로 지정
font_path = "c:/Windows/Fonts/malgun.ttf"  # 윈도우에 설치된 맑은 고딕 폰트 경로

# 폰트 이름 얻어오기
font_name = font_manager.FontProperties(fname=font_path).get_name()

# matplotlib의 rc(run command) 기능을 이용하여 글꼴 설정
mpl.rc('font', family=font_name)

# 유니코드에서  음수 부호 설정
mpl.rc('axes', unicode_minus=False)

df_guest_prefer = pd.read_csv('../../../../../datasets/paris_prefer.csv')
df_non_guest_prefer= pd.read_csv('../../../../../datasets/paris_non_prefer.csv')

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from matplotlib import font_manager, rc
%matplotlib inline
import folium
from folium.plugins import MarkerCluster
from folium import FeatureGroup, LayerControl
from folium.features import GeoJson
from shapely.geometry import shape, Point
import json
from urllib.parse import urlparse, urlunparse, parse_qs, urlencode

# URL에 쿼리 파라미터를 추가하는 함수
def modify_url(url, params):
    url_parts = list(urlparse(url))
    query = dict(parse_qs(url_parts[4]))
    query.update(params)
    url_parts[4] = urlencode(query, doseq=True)
    return urlunparse(url_parts)

# 글꼴 경로 지정
font_path = "c:/Windows/Fonts/malgun.ttf"  # 윈도우에 설치된 맑은 고딕 폰트 경로

# 폰트 이름 얻어오기
font_name = font_manager.FontProperties(fname=font_path).get_name()

# matplotlib의 rc(run command) 기능을 이용하여 글꼴 설정
mpl.rc('font', family=font_name)

# 유니코드에서  음수 부호 설정
mpl.rc('axes', unicode_minus=False)

path = '../../../../../datasets/london_listings.csv'

raw = pd.read_csv(path)
df = raw.copy()

df = df[['id', 'host_id','host_is_superhost',
        'host_total_listings_count','neighbourhood_cleansed','room_type','accommodates','bathrooms','bedrooms',
        'beds','amenities','price','minimum_nights','maximum_nights','number_of_reviews', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy', 
        'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 
        'review_scores_location', 'review_scores_value', 'reviews_per_month','listing_url','property_type','number_of_reviews_ltm','has_availability','last_review']]

df = df.rename(columns= {
    'id': '숙소_id',
    'host_id': '호스트_id',
    'host_is_superhost': '슈퍼호스트',
    'host_total_listings_count': '숙소_수',
    'neighbourhood_cleansed': '숙소_지역',
    'room_type': '숙소_유형',
    'accommodates': '수용_인원수',
    'bathrooms': '욕실수',
    'bedrooms': '침실수',
    'beds': '침대수',
    'amenities': '편의시설',
    'price': '숙소_가격',
    'minimum_nights': '최소_숙박일',
    'maximum_nights': '최대_숙박일',
    'number_of_reviews': '리뷰수',
    'number_of_reviews_l30d': '30일_리뷰수',
    'review_scores_rating': '리뷰점수',
    'review_scores_accuracy': '숙소_정확성_리뷰점수',
    'review_scores_cleanliness': '숙소_청결도_리뷰점수',
    'review_scores_checkin': '숙소_체크인_리뷰점수',
    'review_scores_communication': '숙소_소통_리뷰점수',
    'review_scores_location': '숙소_위치_리뷰점수',
    'review_scores_value': '숙소_가격_리뷰점수',
    'reviews_per_month': '평균_리뷰수',
    'listing_url':'url',
    'property_type':'숙소_특징',
    'number_of_reviews_ltm':'12개월_리뷰수',
    'has_availability':'예약가능여부',
    'last_review':'마지막_리뷰'
})
df['위도']=raw['latitude']
df['경도']=raw['longitude']
print(len(df[df['리뷰수']==0]))
# 슈퍼호스트, 리뷰수 결측치 제거
print(f"처음 df : {len(df)}")
df = df[~df['슈퍼호스트'].isnull()]
print(f"슈퍼호스트 제거 : {len(df)}")
df = df[df['리뷰수']>2]
print(f"리뷰수 제거 : {len(df)}")

# 숙소가격 null값 제거
df= df[~df['숙소_가격'].isnull()]

#가격 앞 통화기호 제거
df['숙소_가격'] = df['숙소_가격'].replace('[\$,]', '', regex=True).astype(float)

# 유형 제거 
df = df[(df['숙소_유형']=='Entire home/apt') | (df['숙소_유형']=='Private room')]
print(f"숙소_유형 제거 : {len(df)}")

# 12개월 리뷰수 0 개 제거
df = df[df['12개월_리뷰수']!=0]
print(f"12개월_리뷰수 0개 제거 : {len(df)}")

# 욕실수, 침실수, 침대수 null값 제거
df = df.dropna(subset=['욕실수', '침실수', '침대수'])
print(f"욕실수, 침실수, 침대수 제거 후 : {len(df)}")

# 리뷰  null값 제거
df = df.dropna(subset=['숙소_정확성_리뷰점수','숙소_청결도_리뷰점수','숙소_체크인_리뷰점수','숙소_소통_리뷰점수','숙소_위치_리뷰점수','숙소_가격_리뷰점수'])
print(f"리뷰 결측치 제거 후 : {len(df)}")

# 예약 가능여부 f 버리기
df = df.dropna(subset='예약가능여부')
print(f"예약가능여부 : {len(df)}")

df_guest_prefer= df[(df['리뷰점수'] >= 4.9) & (df['리뷰수'] >= 5) & (df['슈퍼호스트'] == 't')]
df_non_guest_prefer = df[(df['슈퍼호스트'] == 'f') & (df['리뷰수'] >= 5)].sort_values('리뷰점수', ascending=True).head(4695)

24983
처음 df : 90852
슈퍼호스트 제거 : 90463
리뷰수 제거 : 50711
숙소_유형 제거 : 34776
12개월_리뷰수 0개 제거 : 31852
욕실수, 침실수, 침대수 제거 후 : 31781
리뷰 결측치 제거 후 : 31781
예약가능여부 : 31779


In [23]:

df_station = pd.read_csv('./런던_data/London_stations.csv')

In [24]:
df_station = df_station[df_station['NETWORK']=='London Underground'][['NAME','x','y']]

In [25]:
df_station = df_station.rename(columns = {
    'NAME' : '역이름',
    'x' :'경도',
    'y' : '위도' 
})

In [26]:
df_station.isnull().sum()

역이름    0
경도     0
위도     0
dtype: int64

In [46]:
import folium
from folium.plugins import MarkerCluster
from folium import FeatureGroup, LayerControl

# 런던 중심부의 위도와 경도
london_latitude = 51.5074
london_longitude = -0.1278

# 런던 지도 생성
map_london = folium.Map(location=[london_latitude, london_longitude], zoom_start=12)

# 랜드마크 위치 데이터
landmarks = {
    "런던 아이": (51.5033, -0.1196),
    "타워 브리지": (51.5055, -0.0754),
    "버킹엄 궁전": (51.5014, -0.1419),
    "대영 박물관": (51.5194, -0.1270),
    "테이트 모던": (51.5076, -0.0994),
    "세인트 폴 대성당": (51.5138, -0.0984),
    "웨스트민스터 사원": (51.4993, -0.1273),
    "피카딜리 서커스": (51.5101, -0.1342),
    "하이드 파크": (51.5073, -0.1657),
    "캠든 마켓": (51.5416, -0.1469),
    "빅벤": (51.5007, -0.1246),
    "내셔널 갤러리": (51.5089, -0.1283),
    "자연사 박물관": (51.4967, -0.1764),
    "타워 오브 런던": (51.5081, -0.0759),
    "코벤트 가든": (51.5115, -0.1234),
    "리젠트 파크": (51.5313, -0.1569),
    "셔드": (51.5045, -0.0865),
    "킹스 크로스 세인트 판크라스": (51.5308, -0.1238)
}

# 런던 경계 데이터 로드 및 추가
london_boundary_geojson = './런던_data/London_boundary.geojson'
GeoJson(
    london_boundary_geojson,
    name='London Boundary',
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1
    }
).add_to(map_london)

# 랜드마크에 마커 추가
for landmark, (lat, lng) in landmarks.items():
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='star', prefix='fa', color='blue'),
        popup=landmark
    ).add_to(map_london)

# 클러스터 객체 생성 및 사용자 데이터 위치 표시
marker_cluster = MarkerCluster().add_to(map_london)
for idx, row in df_guest_prefer.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(color='green', icon='info-sign'),
        popup='Guest Prefer'
    ).add_to(marker_cluster)

for idx, row in df_non_guest_prefer.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(color='red', icon='info-sign'),
        popup='Non-Guest Prefer'
    ).add_to(marker_cluster)

# 지하철 레이어 추가
subway_layer = FeatureGroup(name='Subway Stations', show=False)
for idx, row in df_station.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        icon=folium.Icon(icon='subway', prefix='fa', color='black'),
        popup=f"{row['역이름']} Station"
    ).add_to(subway_layer)
subway_layer.add_to(map_london)

# 레이어 컨트롤 추가
LayerControl().add_to(map_london)

# 지도 저장
map_london.save('london_map.html')


In [29]:
import pandas as pd
from geopy.distance import geodesic

# 각 역에서 500m 이내에 있는 df_guest_prefer와 df_non_guest_prefer의 개수를 세는 함수
def count_nearby(df, station_lat, station_lon, radius=0.5):
    count = 0
    if pd.isna(station_lat) or pd.isna(station_lon):
        return count  # 입력된 역의 위도 또는 경도가 NaN이면 계산을 수행하지 않고 0을 반환
    station_coords = (station_lat, station_lon)
    for idx, row in df.iterrows():
        if pd.isna(row['위도']) or pd.isna(row['경도']):
            continue  # 데이터프레임의 행에 NaN이 있으면 이 행을 건너뛰기
        point_coords = (row['위도'], row['경도'])
        if geodesic(station_coords, point_coords).km <= radius:
            count += 1
    return count

# 결과를 저장할 리스트
results = []

# df_station_check의 각 역에 대해 반복
for idx, station in df_station.iterrows():
    station_name = station['역이름']
    lat = station['위도']
    lon = station['경도']
    
    # df_guest_prefer와 df_non_guest_prefer에서 각각의 카운트를 가져옴
    guest_count = count_nearby(df_guest_prefer, lat, lon)
    non_guest_count = count_nearby(df_non_guest_prefer, lat, lon)
    
    # 결과 저장
    results.append({
        '역이름': station_name,
        '위도': lat,
        '경도': lon,
        '게스트_선호': guest_count,
        '게스트_불호': non_guest_count
    })

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(results)

# 결과 출력
print(result_df)

                    역이름         위도        경도  게스트_선호  게스트_불호
0                Temple  51.510474 -0.112644       4       8
1           Blackfriars  51.511114 -0.102020       2       9
2         Mansion House  51.511306 -0.092495       3      11
3         Cannon Street  51.510963 -0.088801       2      19
4              Monument  51.510209 -0.084502       3      14
..                  ...        ...       ...     ...     ...
266             Croxley  51.646588 -0.440192       0       0
267             Watford  51.656992 -0.415833       0       0
268  Chalfont & Latimer  51.667546 -0.559160       0       0
269             Chesham  51.704742 -0.609421       0       0
270            Amersham  51.673662 -0.606147       0       0

[271 rows x 5 columns]


In [30]:
len(df_guest_prefer)

4695

In [31]:
len(df_non_guest_prefer)

4695

In [32]:
result_df['게스트_선호'].sum()

2298

In [33]:
result_df['게스트_불호'].sum()

3888

In [39]:
from geopy.distance import geodesic

# 특정 랜드마크와의 거리 계산 함수 정의
def count_accommodations_within_distance(df, landmark_coords, max_distance_km):
    count = 0
    for idx, row in df.iterrows():
        # 숙소 위치
        accommodation_location = (row['위도'], row['경도'])
        # 랜드마크와 숙소 간의 거리 계산
        distance = geodesic(landmark_coords, accommodation_location).km
        # 특정 거리 이내에 있으면 카운트
        if distance <= max_distance_km:
            count += 1
    return count

result1 = []
# 각 랜드마크와의 거리 계산
landmark_distances = {}
for landmark, coords in landmarks.items():
    count_prefer = count_accommodations_within_distance(df_guest_prefer, coords, 0.5)  # 1km 이내
    count_non_prefer = count_accommodations_within_distance(df_non_guest_prefer, coords, 0.5)  # 1km 이내
    landmark_distances[landmark] = {'Guest Prefer': count_prefer, 'Non-Guest Prefer': count_non_prefer}
    # 결과 저장
    result1.append({
        '랜드마크': landmark,
        '위도': coords[0],
        '경도': coords[1],
        '게스트_선호': count_prefer,
        '게스트_불호': count_non_prefer
    })
    
# 결과를 데이터프레임으로 변환
result1_df = pd.DataFrame(result1)

# 결과 출력
print(result1_df)


               랜드마크       위도      경도  게스트_선호  게스트_불호
0             런던 아이  51.5033 -0.1196       1       0
1            타워 브리지  51.5055 -0.0754       5       9
2            버킹엄 궁전  51.5014 -0.1419       4      10
3            대영 박물관  51.5194 -0.1270      27      25
4            테이트 모던  51.5076 -0.0994       9       8
5         세인트 폴 대성당  51.5138 -0.0984       2      15
6         웨스트민스터 사원  51.4993 -0.1273       5       5
7          피카딜리 서커스  51.5101 -0.1342      31      75
8            하이드 파크  51.5073 -0.1657       0       0
9             캠든 마켓  51.5416 -0.1469      15      24
10               빅벤  51.5007 -0.1246       2       1
11          내셔널 갤러리  51.5089 -0.1283      35      61
12          자연사 박물관  51.4967 -0.1764      19      21
13         타워 오브 런던  51.5081 -0.0759       5      14
14           코벤트 가든  51.5115 -0.1234      53      45
15           리젠트 파크  51.5313 -0.1569       0       0
16               셔드  51.5045 -0.0865       8      23
17  킹스 크로스 세인트 판크라스  51.5308 -0.1238      10  

In [40]:
result1_df['게스트_선호'].sum()

231

In [41]:
result1_df['게스트_불호'].sum()

407